In [2]:
import pandas as pd
import numpy as np

In [20]:
df = pd.read_csv('../data/df_clean.csv', sep=',')

# Erstelle eine Kopie des Dataframes
df_copy = df.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17803 entries, 0 to 17802
Data columns (total 46 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   LeagueCountry                17803 non-null  object 
 1   League                       17803 non-null  object 
 2   NationalLeagueLevel          17803 non-null  object 
 3   Club                         17803 non-null  object 
 4   No                           17803 non-null  float64
 5   Name                         17803 non-null  object 
 6   Value                        17803 non-null  float64
 7   HighestValue                 17803 non-null  float64
 8   Age                          17800 non-null  float64
 9   Height                       16713 non-null  float64
 10  Nationality                  17803 non-null  object 
 11  Foot                         17803 non-null  object 
 12  Position                     17803 non-null  object 
 13  Consultancy     

In [21]:
# Liste der Metadatenspalten
metadata_columns = [
    'LeagueCountry', 'League', 'NationalLeagueLevel', 'Club', 'No', 'Name', 'Value', 'Age', 'Height', 'Nationality', 'Foot', 'Position', 'Consultancy', 'Supplier', 'ClubSince', 'ContractUntil', 'Injury', 'InternationalTeam', 'ActiveInternational', 'FormerInternational', 'Instagram', 'PositionCategory', 'NationalLeagueLevelCategory', 'Trending'
]

# Erstellen Sie einen neuen DataFrame, der nur Metadaten enthält
metadata_df = df[metadata_columns]

# Zeigen Sie die Informationen zum neuen DataFrame an
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17803 entries, 0 to 17802
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   LeagueCountry                17803 non-null  object 
 1   League                       17803 non-null  object 
 2   NationalLeagueLevel          17803 non-null  object 
 3   Club                         17803 non-null  object 
 4   No                           17803 non-null  float64
 5   Name                         17803 non-null  object 
 6   Value                        17803 non-null  float64
 7   Age                          17800 non-null  float64
 8   Height                       16713 non-null  float64
 9   Nationality                  17803 non-null  object 
 10  Foot                         17803 non-null  object 
 11  Position                     17803 non-null  object 
 12  Consultancy                  17803 non-null  object 
 13  Supplier        

In [22]:
metadata_df = pd.get_dummies(metadata_df, drop_first=True)
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17803 entries, 0 to 17802
Columns: 21569 entries, No to PositionCategory_Torwart
dtypes: float64(6), int64(5), uint8(21558)
memory usage: 367.5 MB


In [19]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder-Objekt erzeugen
le = LabelEncoder()

# Liste aller Spalten, die kategorische Variablen enthalten
cat_cols = ['LeagueCountry', 'League', 'NationalLeagueLevel', 'Club', 'Nationality', 'Foot', 'Position', 'Consultancy', 'Supplier', 'InternationalTeam', 'Instagram', 'PositionCategory']


# Jede Spalte in cat_cols label encoden
for col in cat_cols:
    if metadata_df[col].dtype == 'object':  # nur Spalten vom Typ 'object' label encoden
        metadata_df[col] = le.fit_transform(df[col].astype(str))

C:\Users\marc_\AppData\Local\Temp\ipykernel_2896\153291852.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_df[col] = le.fit_transform(df[col].astype(str))


In [23]:
# X und y festlegen
y = metadata_df["Value"]
X = metadata_df.drop("Value", axis = 1)

In [24]:
from sklearn.model_selection import train_test_split
# Test- und Trainingsdaten Aufteilung
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# Import für K-Nearest-Neighbor-Imputer
from sklearn.impute import KNNImputer

# K-Nearest-Neighbor-Imputer verwenden um fehlende Werte in den Trainingsdaten zu ersetzen
imputer = KNNImputer()
imputer.fit(X_train)
X_train_imputed = imputer.transform(X_train)

In [ ]:
# Namen der Features des imputierten Datensatzes bekommen
names = imputer.feature_names_in_

In [ ]:
# Dataframe aus den imputierten Daten erstellen
X_train_imputed = pd.DataFrame(X_train_imputed, columns=names, index = X_train.index)
X_train_imputed.info()

In [ ]:
# Imputation für die Traingsdaten separat vornehmen
X_test_imputed = imputer.transform(X_test)
X_test_imputed = pd.DataFrame(X_test_imputed, columns = names, index=X_test.index)
X_test_imputed.info()

In [ ]:
import matplotlib.pyplot as plt
from pandas import DataFrame
# Kopiert aus TreesForestsEnsembles.ipynb
def plot_variable_importance(model, X_train):
    imp=DataFrame({"imp":model.feature_importances_, "names":X_train.columns}).sort_values("imp", ascending=True)
    fig, ax = plt.subplots(figsize=(imp.shape[0]/6,imp.shape[0]/5), dpi=300)
    ax.barh(imp["names"],imp["imp"], color="green")
    ax.set_xlabel('\nVariable Importance')
    ax.set_ylabel('Features\n')
    ax.set_title('Variable Importance Plot\n')
    plt.show()

In [ ]:
# Import Functions
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Instantiate Model
model = XGBRegressor()

# fit model
model.fit(X_train, y_train)

#make prediction
y_pred = model.predict(X_test)

# Evaluate Model Performance
mean_squared_error(y_test, y_pred, squared=False)

In [ ]:
plot_variable_importance(model, X_train)